# Compare Models

This notebook compares various GFW models based on the `measure_speed` and `measure_course` with each other
and with the models from Dalhousie University.  Note that the distance-to-shore cutoff was disabled in the
Dalhousie models, so none of the models compared here are using distance-to-shore as a feature.

In [3]:
from __future__ import print_function, division
%matplotlib inline
import sys
import numpy as np
sys.path.append('..')
import warnings; warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML, Markdown
import datetime
import pytz
from sklearn import metrics

import vessel_scoring.models
import vessel_scoring.evaluate_model
from vessel_scoring import utils

In [4]:
data = vessel_scoring.models.load_data('../datasets')

Warning, insufficient items to sample, returning all
Warning, insufficient items to sample, returning all


In [43]:
import vessel_scoring.data
from glob import glob
import os
paths = glob("../datasets/from_ranges/*.npz")
print(paths)
rngdata = {os.path.basename(p) : vessel_scoring.data.load_dataset_by_vessel(p) for p in paths
          if p.endswith('.measures.npz')}


['../datasets/from_ranges/split_Longliners.csv.npz', '../datasets/from_ranges/split_Longliners.measures.npz', '../datasets/from_ranges/split_Pole_and_Line.csv.npz', '../datasets/from_ranges/split_Pole_and_Line.measures.npz', '../datasets/from_ranges/split_Pots_and_Traps.csv.npz', '../datasets/from_ranges/split_Pots_and_Traps.measures.npz', '../datasets/from_ranges/split_Purse_seines.csv.npz', '../datasets/from_ranges/split_Purse_seines.measures.npz', '../datasets/from_ranges/split_Set_gillnets.csv.npz', '../datasets/from_ranges/split_Set_gillnets.measures.npz', '../datasets/from_ranges/split_Trawlers.csv.npz', '../datasets/from_ranges/split_Trawlers.measures.npz', '../datasets/from_ranges/split_Trollers.csv.npz', '../datasets/from_ranges/split_Trollers.measures.npz']
Warning, insufficient items to sample, returning all
Warning, insufficient items to sample, returning all
Warning, insufficient items to sample, returning all
Warning, insufficient items to sample, returning all


In [6]:
for gear in rngdata:
    print(gear)

    orig_mmsi = sorted(set(data["kristina_{}".format(gear)]['all']['mmsi']))

    rngdata_mmsi = sorted(set(rngdata[gear]['mmsi']))

    common = sorted(set(orig_mmsi) & set(rngdata_mmsi))

    def plot(x, ax):
        mask = utils.is_fishy(x)
        ax.plot(x[~mask]['lon'], x[~mask]['lat'], 'b.')
        ax.plot(x[mask]['lon'], x[mask]['lat'], 'r.')



    import matplotlib.pyplot as plt
    def plot_stuff(n):
        mmsi = common[n]
        rngd = rngdata[gear][rngdata[gear]['mmsi'] == mmsi]
        origd = data["kristina_{}".format(gear)]['all'][data["kristina_{}".format(gear)]['all']['mmsi'] == mmsi]
        fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(16, 4))
        plot(origd, ax1)
        plot(rngd, ax2)
        ax2.set_xlim(*ax1.get_xlim())
        ax2.set_ylim(*ax1.get_ylim())
        plt.show()
        print(len(rngd) / len(origd) if len(origd) else None, len(rngd), len(origd))


    for i in range(len(common)):
        plot_stuff(i)



split_Pots_and_Traps.measures.npz


KeyError: 'kristina_split_Pots_and_Traps.measures.npz'

In [7]:
text = """
Pairwise agreement for Longliners 0.86721601765
Consensus agreement for Longliners 0.933060883222
Standard Deviationn for Longliners 0.0824210430256

Pairwise agreement for Pole and Line 0.953205472904
Consensus agreement for Pole and Line 0.975634821055
Standard Deviationn for Pole and Line 0.029320597482

Pairwise agreement for Pots and Traps 0.984735367311
Consensus agreement for Pots and Traps 0.992367683656
Standard Deviationn for Pots and Traps 0.0101241072179

Pairwise agreement for Purse seines 0.916769633641
Consensus agreement for Purse seines 0.958270409071
Standard Deviationn for Purse seines 0.0513507211783

Pairwise agreement for Set gillnets 0.965576816174
Consensus agreement for Set gillnets 0.982649433863
Standard Deviationn for Set gillnets 0.0225498520952

Pairwise agreement for Trawlers 0.932607775748
Consensus agreement for Trawlers 0.966164556048
Standard Deviationn for Trawlers 0.0418758939906

Pairwise agreement for Trollers 0.946113904902
Consensus agreement for Trollers 0.973056952451
Standard Deviationn for Trollers 0.0319558123579"""

agreement = {}
primer = "Pairwise agreement for "
for line in text.split("\n"):
    line = line.strip()
    if not line:
        continue
    if line.startswith(primer):
        line = line[len(primer):]
        name, strval = line.rsplit(" ", 1)
        val = float(strval)
        agreement[name] = val
        
agreement

{'Longliners': 0.86721601765,
 'Pole and Line': 0.953205472904,
 'Pots and Traps': 0.984735367311,
 'Purse seines': 0.916769633641,
 'Set gillnets': 0.965576816174,
 'Trawlers': 0.932607775748,
 'Trollers': 0.946113904902}

In [37]:
fishing_localization = """
<div class="unbreakable">
  <table>
    <tr>
      <th>
        Gear Type
      </th>
      <th>
        Precision
      </th>
      <th>
        Recall
      </th>
      <th>
        Accuracy
      </th>
      <th>
        F1-Score
      </th>
    </tr>
    <tr>
      <td>
        Longliners
      </td>
      <td>
        0.85
      </td>
      <td>
        0.82
      </td>
      <td>
        0.85
      </td>
      <td>
        0.83
      </td>
    </tr>
    <tr>
      <td>
        Pole and Line
      </td>
      <td>
        0.91
      </td>
      <td>
        0.95
      </td>
      <td>
        0.96
      </td>
      <td>
        0.93
      </td>
    </tr>
    <tr>
      <td>
        Pots and Traps
      </td>
      <td>
        0.98
      </td>
      <td>
        0.97
      </td>
      <td>
        0.98
      </td>
      <td>
        0.97
      </td>
    </tr>
    <tr>
      <td>
        Purse seines
      </td>
      <td>
        0.83
      </td>
      <td>
        0.87
      </td>
      <td>
        0.90
      </td>
      <td>
        0.85
      </td>
    </tr>
    <tr>
      <td>
        Set gillnets
      </td>
      <td>
        0.97
      </td>
      <td>
        0.86
      </td>
      <td>
        0.93
      </td>
      <td>
        0.91
      </td>
    </tr>
    <tr>
      <td>
        Trawlers
      </td>
      <td>
        0.94
      </td>
      <td>
        0.72
      </td>
      <td>
        0.87
      </td>
      <td>
        0.81
      </td>
    </tr>
    <tr>
      <td>
        Trollers
      </td>
      <td>
        0.94
      </td>
      <td>
        0.52
      </td>
      <td>
        0.74
      </td>
      <td>
        0.67
      </td>
    </tr>
    <tr>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>
        Overall
      </td>
      <td>
        0.88
      </td>
      <td>
        0.80
      </td>
      <td>
        0.87
      </td>
      <td>
        0.84
      </td>
    </tr>
  </table>
</div>
"""

In [89]:
# Comparison function:
#   1. Convert output to ranges
#   2. Convert by minutes

import sys
sys.path.append("../../mussidae")
import mussidae.time_range_tools as trtools
sys.path.append("../../vessel-classification-pipeline/classification")
from  classification.utility import is_test




def compare_fishing_localization(true_ranges, inferred_ranges):

    all_mmsi = sorted(set(x.mmsi for x in true_ranges))

    true_by_mmsi = {}
    pred_by_mmsi = {}

    for mmsi in all_mmsi:
        true = np.array([x for x in true_ranges if x.mmsi == mmsi])
        inferred = np.array([x for x in inferred_ranges if x.mmsi == mmsi])

        # Determine minutes from start to finish of this mmsi, create an array to
        # hold results and fill with -1 (unknown)
        _, start, end, _ = true[0]
        for (_, s, e, _) in true[1:]:
            start = min(start, s)
            end = max(end, e)
        start_min = datetime_to_minute(start)
        end_min = datetime_to_minute(end)
        
        minutes = np.empty([end_min - start_min + 1, 2], dtype=int)
        minutes.fill(-1)

        # Fill in minutes[:, 0] with known true / false values
        for (_, s, e, is_fishing) in true:
            s_min = datetime_to_minute(s)
            e_min = datetime_to_minute(e)
            for m in range(s_min - start_min, e_min - start_min + 1):
                minutes[m, 0] = is_fishing

        # fill in minutes[:, 1] with in inferred values
        for (_, s, e, is_fishing) in inferred:
            s_min = datetime_to_minute(s)
            e_min = datetime_to_minute(e)
            for m in range(s_min - start_min, e_min - start_min + 1):
                if 0 <= m < len(minutes):
                    minutes[m, 1] = is_fishing       
 
        mask = ((minutes[:, 0] != -1) & (minutes[:, 1] != -1))

        if mask.sum():
            true_by_mmsi[mmsi] = minutes[mask, 0]
            pred_by_mmsi[mmsi] = minutes[mask, 1]
            
    return true_by_mmsi, pred_by_mmsi



def ranges_from_ais(ais, is_fishing):
    points = [trtools.Point(x['mmsi'], 
                            datetime.datetime.utcfromtimestamp(x['timestamp']).replace(tzinfo=pytz.utc), is_fishing[i])
              for i, x in enumerate(ais)]
    return trtools.ranges_from_points(points)

def datetime_to_minute(dt):
    timestamp = (dt - datetime.datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds()
    return int(timestamp // 60)


def model_true_inferred(mdl, test_data):
    check =  mdl.predict_proba(test_data)[:,1]
    
    inferred_ranges = list(ranges_from_ais(test_data, mdl.predict_proba(test_data)[:,1] > 0.5))
    true_ranges = list(ranges_from_ais(test_data, test_data['classification'] > 0.5))
    
    true, inferred = compare_fishing_localization(true_ranges, inferred_ranges)
    
    y_true = np.concatenate(true.values())
    y_pred = np.concatenate(inferred.values())
    
    return y_true, y_pred


def model_metrics(y_true, y_pred):
    
    accuracy = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    
    return precision, recall, accuracy, f1, y_true, y_pred
    

#     lines = ["|Model|Recall|Precision|F1-Score|Accuracy|",
#          "|-----|------|---------|--------|"]


#     for name in sorted(predictions):
#         pred, actual = predictions[name]
#         lines.append("|{}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|".format(
#                 name, 
#                 metrics.recall_score(actual, pred),
#                 metrics.precision_score(actual, pred), 
#                 metrics.f1_score(actual, pred),
#                 metrics.accuracy_score(actual, pred)))
#     return '\n'.join(lines)



In [91]:
# To get test data split by *predicted* class:
import csv

# Concatenate all test data
all_data = np.concatenate([x[0] for x in rngdata.values()])

# Create a mapping of true MMSI to class
true_label_map = {}
for gear_type in rngdata:
    gear_name = os.path.splitext(os.path.splitext(gear_type)[0])[0][6:]
    for x in rngdata[gear_type][0]:
        true_label_map[x['mmsi']] = gear_name

# Load a mapping of inferred MMSI to class
inferred_label_map = {}
with open("../../vessel-classification-pipeline copy/classification/inferred_labels.csv") as f:
    for row in csv.DictReader(f):
        inferred_label_map[int(row['mmsi'])] = row['label'].strip().replace('/', '_').replace(' ', '_')

testrngdata = {}
    
for gear_type in rngdata:
    gear_name = os.path.splitext(os.path.splitext(gear_type)[0])[0][6:]
    mask = [(inferred_label_map.get(int(x['mmsi'])) == gear_name and is_test(int(x['mmsi']))) for x in all_data]
    mask = np.array(mask)
    if mask.sum():
        testrngdata[gear_type] = all_data[mask]

# predict all results (below)

# Split on true label map
# TODO

In [106]:
models = {
#     'Random Forest' :vessel_scoring.random_forest_model.RandomForestModel(
#         colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#                                     measures=['speed'])), 
        'Random Forest with Distances' :vessel_scoring.random_forest_model.RandomForestModel(
                colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
                                            measures=['speed', 'distance_from_shore', 'distance_from_port'])),
#         'Logistic' :vessel_scoring.logistic_model.LogisticModel(order=6,
#                 colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#                                             measures=['measure_speed'])),
#         'Logistic with Distances' :vessel_scoring.logistic_model.LogisticModel(order=6,
#                 colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#                                             measures=['measure_speed', 'measure_distance_from_shore', 
#     'measure_distance_from_port'])),                                                               
#         'LogisticX' :vessel_scoring.logistic_model.LogisticModel(order=6, cross=4,
#                 colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#                                             measures=['measure_speed'])),
#         'LogisticX with Distances' :vessel_scoring.logistic_model.LogisticModel(order=6, cross=4,
#                 colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#   measures=['measure_speed', 'measure_distance_from_shore', 'measure_distance_from_port'])),                                                         
                                                               } 

lines = ["|Model|Count|Points|Precision|Recall|Accuracy|F1-Score|",
         "|-----|-----|------|---------|------|--------|--------|"]

all_true = []
all_pred = []
all_mmsi = []

for gear in sorted(rngdata):
    title = gear.split('.')[0].split('_',1)[1].replace('_', ' ')
        
    gear_name = os.path.splitext(os.path.splitext(gear)[0])[0][6:]
    
    print("Computing", gear_name)
    
    all, _, _, _ = rngdata[gear]
    all['classification'] = all['classification'] > 0.5

    train  = []
    test = []
    for x in all:
        if not is_test(int(x['mmsi'])):
            train.append(x)
    train = np.array(train)
    if gear not in testrngdata:
        continue
    test = testrngdata[gear]
    test['classification'] = test['classification'] > 0.5
    
    trained_models = []
    for name, mdl in reversed(sorted(models.items())):
        trained_models.append((name,
                           vessel_scoring.models.train_model_on_data(mdl, train)))

    name, mdl = trained_models[0]
    y_true, y_pred =  model_true_inferred(mdl, test)

    all_mmsi.append(test['mmsi'].astype(int))
    all_true.append(y_true)
    all_pred.append(y_pred)
    
    
mmsi_all = np.concatenate(all_mmsi)
y_true_all = np.concatenate(all_true)
y_pred_all = np.concatenate(all_pred)

for gear_type in sorted(set(true_label_map.values())):
    mask = [(true_label_map.get(x) == gear_type and is_test(x)) for x in mmsi_all]
    mask = np.array(mask)
    y_true = y_true_all[mask]
    y_pred = y_pred_all[mask]
    mmsi_count = len(set(mmsi_all[mask]))
    n_points = len(mmsi_all[mask])
    accuracy = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    lines.append('|{}|{}|{}|'.format(gear_type, mmsi_count, n_points) + '|'.join(['{:.2f}'.format(x) for x in
                                [precision, recall, accuracy, f1]]) + '|')
    
    
mask = np.array([(is_test(x)) for x in mmsi_all])
y_true = y_true_all[mask]
y_pred = y_pred_all[mask]
    
mmsi_count =len(set(mmsi_all[mask]))
n_points = len(mmsi_all[mask])
accuracy = metrics.accuracy_score(y_true, y_pred)
precision = metrics.precision_score(y_true, y_pred)
recall = metrics.recall_score(y_true, y_pred)
f1 = metrics.f1_score(y_true, y_pred)

lines.append("|     |     |     |     |     |")

lines.append('|Overall|{}|{}|'.format(mmsi_count, n_points) + '|'.join(['{:.2f}'.format(x) for x in
                            [precision, recall, accuracy, f1]]) + '|')

display(HTML("<h2>{}</h2>".format(name)))

display(Markdown('\n'.join(lines)))

Computing Longliners
Computing Pole_and_Line
Computing Pots_and_Traps
Computing Purse_seines
Computing Set_gillnets
Computing Trawlers
Computing Trollers


|Model|Count|Points|Precision|Recall|Accuracy|F1-Score|
|-----|-----|------|---------|------|--------|--------|
|Longliners|33|53089|0.85|0.79|0.91|0.82|
|Pole_and_Line|3|3015|0.82|1.00|0.90|0.90|
|Pots_and_Traps|3|1752|0.92|1.00|0.92|0.96|
|Purse_seines|10|10686|0.72|1.00|0.87|0.84|
|Set_gillnets|7|7803|0.65|1.00|0.87|0.79|
|Trawlers|12|17915|0.92|0.96|0.92|0.94|
|Trollers|1|2768|0.79|1.00|0.83|0.88|
|     |     |     |     |     |
|Overall|69|97028|0.84|0.91|0.90|0.87|

In [104]:
print((y_true_all == y_pred_all).mean())

0.923687074465


In [38]:
display(HTML("<h2>Neural Net</h2>"))
display(HTML(fishing_localization))

Gear Type,Precision,Recall,Accuracy,F1-Score
Longliners,0.85,0.82,0.85,0.83
Pole and Line,0.91,0.95,0.96,0.93
Pots and Traps,0.98,0.97,0.98,0.97
Purse seines,0.83,0.87,0.90,0.85
Set gillnets,0.97,0.86,0.93,0.91
Trawlers,0.94,0.72,0.87,0.81
Trollers,0.94,0.52,0.74,0.67
,,,,
Overall,0.88,0.80,0.87,0.84


In [107]:
models = {
#     'Random Forest' :vessel_scoring.random_forest_model.RandomForestModel(
#         colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#                                     measures=['speed'])), 
#         'Random Forest with Distances' :vessel_scoring.random_forest_model.RandomForestModel(
#                 colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#                                             measures=['speed', 'distance_from_shore', 'distance_from_port'])),
        'Logistic' :vessel_scoring.logistic_model.LogisticModel(order=6,
                colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
                                            measures=['measure_speed'])),
#         'Logistic with Distances' :vessel_scoring.logistic_model.LogisticModel(order=6,
#                 colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#                                             measures=['measure_speed', 'measure_distance_from_shore', 
#     'measure_distance_from_port'])),                                                               
#         'LogisticX' :vessel_scoring.logistic_model.LogisticModel(order=6, cross=4,
#                 colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#                                             measures=['measure_speed'])),
#         'LogisticX with Distances' :vessel_scoring.logistic_model.LogisticModel(order=6, cross=4,
#                 colspec=dict(windows=vessel_scoring.colspec.Colspec.windows,
#   measures=['measure_speed', 'measure_distance_from_shore', 'measure_distance_from_port'])),                                                         
                                                               } 

lines = ["|Model|Count|Points|Precision|Recall|Accuracy|F1-Score|",
         "|-----|-----|------|---------|------|--------|--------|"]

all_true = []
all_pred = []
all_mmsi = []

for gear in sorted(rngdata):
    title = gear.split('.')[0].split('_',1)[1].replace('_', ' ')
        
    gear_name = os.path.splitext(os.path.splitext(gear)[0])[0][6:]
    
    print("Computing", gear_name)
    
    all, _, _, _ = rngdata[gear]
    all['classification'] = all['classification'] > 0.5

    train  = []
    test = []
    for x in all:
        if not is_test(int(x['mmsi'])):
            train.append(x)
    train = np.array(train)
    if gear not in testrngdata:
        continue
    test = testrngdata[gear]
    test['classification'] = test['classification'] > 0.5
    
    trained_models = []
    for name, mdl in reversed(sorted(models.items())):
        trained_models.append((name,
                           vessel_scoring.models.train_model_on_data(mdl, train)))

    name, mdl = trained_models[0]
    y_true, y_pred =  model_true_inferred(mdl, test)

    all_mmsi.append(test['mmsi'].astype(int))
    all_true.append(y_true)
    all_pred.append(y_pred)
    
    
mmsi_all = np.concatenate(all_mmsi)
y_true_all = np.concatenate(all_true)
y_pred_all = np.concatenate(all_pred)

for gear_type in sorted(set(true_label_map.values())):
    mask = [(true_label_map.get(x) == gear_type and is_test(x)) for x in mmsi_all]
    mask = np.array(mask)
    y_true = y_true_all[mask]
    y_pred = y_pred_all[mask]
    mmsi_count = len(set(mmsi_all[mask]))
    n_points = len(mmsi_all[mask])
    accuracy = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred)
    lines.append('|{}|{}|{}|'.format(gear_type, mmsi_count, n_points) + '|'.join(['{:.2f}'.format(x) for x in
                                [precision, recall, accuracy, f1]]) + '|')
    
    
mask = np.array([(is_test(x)) for x in mmsi_all])
y_true = y_true_all[mask]
y_pred = y_pred_all[mask]
    
mmsi_count =len(set(mmsi_all[mask]))
n_points = len(mmsi_all[mask])
accuracy = metrics.accuracy_score(y_true, y_pred)
precision = metrics.precision_score(y_true, y_pred)
recall = metrics.recall_score(y_true, y_pred)
f1 = metrics.f1_score(y_true, y_pred)

lines.append("|     |     |     |     |     |")

lines.append('|Overall|{}|{}|'.format(mmsi_count, n_points) + '|'.join(['{:.2f}'.format(x) for x in
                            [precision, recall, accuracy, f1]]) + '|')

display(HTML("<h2>{}</h2>".format(name)))

display(Markdown('\n'.join(lines)))

Computing Longliners
Computing Pole_and_Line
Computing Pots_and_Traps
Computing Purse_seines
Computing Set_gillnets
Computing Trawlers
Computing Trollers


|Model|Count|Points|Precision|Recall|Accuracy|F1-Score|
|-----|-----|------|---------|------|--------|--------|
|Longliners|33|53089|0.80|0.75|0.89|0.78|
|Pole_and_Line|3|3015|0.80|1.00|0.87|0.89|
|Pots_and_Traps|3|1752|0.91|1.00|0.91|0.95|
|Purse_seines|10|10686|0.76|1.00|0.87|0.86|
|Set_gillnets|7|7803|0.50|1.00|0.80|0.67|
|Trawlers|12|17915|0.86|0.93|0.87|0.89|
|Trollers|1|2768|0.79|1.00|0.79|0.88|
|     |     |     |     |     |
|Overall|69|97028|0.79|0.88|0.87|0.84|